In [1]:
import numpy    as np
from numpy.testing._private.utils import decorate_methods
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl
import time

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from catboost              import CatBoostRegressor

data = pd.read_csv(r'data\london_merged.csv')

np.random.seed(0)

#target = data['cnt']
#data = data.drop(['cnt'], axis=1)

#Print data shape
#print(target.shape)
#print(data.shape)

#Take a look at nulls 0 nulls
#print(target.isnull().sum())
#print(data.isnull().sum())

#lets create a 2 new feautures
# Hour time stamp contains the year and the month,
# we will create different columns for each one

data['year'] = data['timestamp'].apply(lambda row: row[:4])
data['month'] = data['timestamp'].apply(lambda row: row.split('-')[2][:2] )
data['hour'] = data['timestamp'].apply(lambda row: row.split(':')[0][-2:] )
'''
print(data['year'])
print(data['month'])
print(data['hour'])
'''
data.drop('timestamp', axis=1, inplace=True)

print(data.shape)


def data_enhancement(data):
    
    gen_data = data
    
    for season in data['season'].unique():
        seasonal_data =  gen_data[gen_data['season'] == season]
        hum_std = seasonal_data['hum'].std()
        wind_speed_std = seasonal_data['wind_speed'].std()
        t1_std = seasonal_data['t1'].std()
        t2_std = seasonal_data['t2'].std()
        
        for i in gen_data[gen_data['season'] == season].index:
            if np.random.randint(2) == 1:
                gen_data['hum'].values[i] += hum_std/10
            else:
                gen_data['hum'].values[i] -= hum_std/10
                
            if np.random.randint(2) == 1:
                gen_data['wind_speed'].values[i] += wind_speed_std/10
            else:
                gen_data['wind_speed'].values[i] -= wind_speed_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t1'].values[i] += t1_std/10
            else:
                gen_data['t1'].values[i] -= t1_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t2'].values[i] += t2_std/10
            else:
                gen_data['t2'].values[i] -= t2_std/10

    return gen_data

print(data.head(3))
gen = data_enhancement(data)
print(gen.shape )

C:\Users\T430\anaconda3\envs\strive\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


(17414, 12)
   cnt   t1   t2   hum  wind_speed  weather_code  is_holiday  is_weekend  \
0  182  3.0  2.0  93.0         6.0           3.0         0.0         1.0   
1  138  3.0  2.5  93.0         5.0           1.0         0.0         1.0   
2  134  2.5  2.5  96.5         0.0           1.0         0.0         1.0   

   season  year month hour  
0     3.0  2015    04   00  
1     3.0  2015    04   01  
2     3.0  2015    04   02  
(17414, 12)


In [2]:
gen

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
0,182,3.379372,1.51169,91.910483,6.890895,3.0,0.0,1.0,3.0,2015,04,00
1,138,3.379372,2.98831,94.089517,5.890895,1.0,0.0,1.0,3.0,2015,04,01
2,134,2.879372,2.01169,97.589517,0.890895,1.0,0.0,1.0,3.0,2015,04,02
3,72,1.620628,1.51169,98.910483,0.890895,1.0,0.0,1.0,3.0,2015,04,03
4,47,1.620628,0.48831,91.910483,5.609105,1.0,0.0,1.0,3.0,2015,04,04
...,...,...,...,...,...,...,...,...,...,...,...,...
17409,1042,5.379372,0.51169,82.089517,19.890895,3.0,0.0,0.0,3.0,2017,03,19
17410,541,5.379372,1.48831,79.910483,20.109105,4.0,0.0,0.0,3.0,2017,03,20
17411,337,5.879372,1.98831,79.589517,23.109105,4.0,0.0,0.0,3.0,2017,03,21
17412,224,5.879372,1.01169,74.910483,23.890895,4.0,0.0,0.0,3.0,2017,03,22


In [38]:
y = data['cnt']
x = data.drop(['cnt'], axis=1)

In [46]:
seasonal_data3 =  data[data['season'] == 3]
hum_std3 = data['hum'].std()
hum_std3

14.38755232457826

In [48]:
def data_enhancement(data):
    
    gen_data = data
    
    for season in data['season'].unique():
        seasonal_data =  gen_data[gen_data['season'] == season]
        hum_std = seasonal_data['hum'].mean()
        wind_speed_std = seasonal_data['wind_speed'].mean()
        t1_std = seasonal_data['t1'].mean()
        t2_std = seasonal_data['t2'].mean()
        
        for i in gen_data[gen_data['season'] == season].index:
            if np.random.randint(2) == 1:
                gen_data['hum'].values[i] += hum_std/10
            else:
                gen_data['hum'].values[i] -= hum_std/10
                
            if np.random.randint(2) == 1:
                gen_data['wind_speed'].values[i] += wind_speed_std/10
            else:
                gen_data['wind_speed'].values[i] -= wind_speed_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t1'].values[i] += t1_std/10
            else:
                gen_data['t1'].values[i] -= t1_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t2'].values[i] += t2_std/10
            else:
                gen_data['t2'].values[i] -= t2_std/10

    return gen_data

print(data.head(3))
# gen = data_enhancement(data)
# print(gen.shape )

   cnt        t1        t2         hum  wind_speed  weather_code  is_holiday  \
0  182  2.610642  0.941361   84.116634    5.174836           3.0         0.0   
1  138  2.610642  2.417980  101.883366    4.174836           1.0         0.0   
2  134  3.648102  2.582020   89.795668    2.606953           1.0         0.0   

   is_weekend  season  year month hour  
0         1.0     3.0  2015    04   00  
1         1.0     3.0  2015    04   01  
2         1.0     3.0  2015    04   02  


In [49]:
print(gen.head(3))

   cnt        t1        t2         hum  wind_speed  weather_code  is_holiday  \
0  182  2.610642  0.941361   84.116634    5.174836           3.0         0.0   
1  138  2.610642  2.417980  101.883366    4.174836           1.0         0.0   
2  134  3.648102  2.582020   89.795668    2.606953           1.0         0.0   

   is_weekend  season  year month hour  
0         1.0     3.0  2015    04   00  
1         1.0     3.0  2015    04   01  
2         1.0     3.0  2015    04   02  
